In [ ]:
import promptflow.azure as pf 

# configure azureml workspace ml_client
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

client = MLClient(
    credential=DefaultAzureCredential(),
    subscription_id="96aede12-2f73-41cb-b983-6d11a904839b",
    resource_group_name="promptflow",
    workspace_name="promptflow-eastus",
)
pf.configure(client=client)

In [ ]:
# load flow
local_file = "./web_classification/"
flow = pf.load_flow(source=local_file)

In [ ]:
print(flow)

In [ ]:
# standard flow using remote data
run = flow.submit_bulk_run(
    data="./webClassification20.csv",
    connections={},
    runtime="wanhan-mir",
)

In [ ]:
run.wait_for_completion()

In [ ]:
from promptflow.azure.entities._bulk_flow_run_input import BulkFlowRunInput


classification_accuracy_eval = pf.load_flow("azureml://flows/QnARelevanceEvaluation")

bulk_flow_run_input = BulkFlowRunInput(
    data="./webClassification20.csv",
    variants=[run],
    inputs_mapping={"question": "data.url", "answer": "data.answer", "context": "data.evidence"},
)

baseline_accuracy = classification_accuracy_eval.submit_bulk_run(
    data=bulk_flow_run_input,
    runtime="wanhan-mir",
)

print(baseline_accuracy)

In [ ]:
baseline_accuracy.wait_for_completion()